In [1]:
# STANDARD LIBRARY IMPORTS
import re
import csv
import os
import datetime
import subprocess
import sys # for args

# THIRD PARTY IMPORTS
import praw

# PROJECT IMPORTS
from setup import reddit
from settings import *
from tools import *
from inbox import check_messages
from reminders import send_reminders
from log_missed_comment import manually_award_comment

subreddit = reddit.subreddit(TARGET_SUBREDDIT)

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.
Loaded 39 users who have opted out from follow-ups.
Loaded 29 users who have opted out from reminders.


Version 7.5.0 of praw is outdated. Version 7.8.1 was released Friday October 25, 2024.


# Step 1: Get the last awarded post

In [2]:
# get the last awarded post. We don't need to look back past this time.
last_award = next( reddit.user.me().comments.new() )

In [3]:
print( last_award.permalink )

/r/photocritique/comments/1o3g2w2/what_do_you_see/njb3a88/


# Step 2: Search the most recent 1,000 comments, and record their IDs

In [4]:
num_processed_comments = 0
num_awarded_comments   = 0

# we can only go back 1,000 comments :|
for awarding_comment in subreddit.comments(limit=None):
    num_processed_comments += 1
    
    if (awarding_comment.author is not None and awarding_comment.author.name != 'AutoModerator') and comment_contains_token( awarding_comment ):
        print(f'Should we award points based on comment {awarding_comment.id} by u/{awarding_comment.author.name}?')
        num_awarded_comments += 1

    if awarding_comment.created_utc <= last_award.created_utc:
        # we've gone back far enough
        print('Yay! There were less than 1,000 comments since the bot crashed.\n')
        break
        
print(f'\n\nSearched {num_processed_comments} comments. Found {num_awarded_comments} that might need awarding.')
        
print(f'There is a gap of {( awarding_comment.created_utc - last_award.created_utc ) / 60 / 60:.1f} hours between the last comment we searched, and the last award.')
print('Hopefully the above number is 0 or small!')

Should we award points based on comment njp9fgo by u/tilltheendof-time?
Should we award points based on comment njo5jzo by u/EmployNo1234?
Should we award points based on comment njnu3xv by u/EmployNo1234?
Should we award points based on comment njlhnl0 by u/There_And_Back1?
Should we award points based on comment njlhfux by u/There_And_Back1?
Should we award points based on comment njlat3d by u/There_And_Back1?
Should we award points based on comment njl2zrj by u/There_And_Back1?
Should we award points based on comment njl10dk by u/There_And_Back1?
Should we award points based on comment njjffqm by u/tuomas_samuli_photos?
Should we award points based on comment nji1nng by u/Present-Vegetable186?
Should we award points based on comment njh6wj5 by u/Present-Vegetable186?
Should we award points based on comment njgcqtj by u/Present-Vegetable186?
Should we award points based on comment njg2i3d by u/tengopelograsoso?
Should we award points based on comment njg2gy2 by u/tengopelograsoso?
Sh

# Step 3: Manually award all the comments above.

Reddit will rate limit you at some point, so you'll need to wait a few mins if you have more than ~15 comments to award.

To award comments, copy the ID above into the below string, and run the cell. Repeat for all comments listed above. You may want to manually check things like:

- don't award to top level comments? 

Are these things still in `run.py`?

In [7]:
from time import sleep

In [8]:
# all as of 18:13 Tues
missing_awards = [
'njp9fgo',
'njo5jzo',
'njnu3xv',
'njlhnl0',
'njlhfux',
'njlat3d',
'njl2zrj',
'njl10dk',
'njjffqm',
'nji1nng',
'njh6wj5',
'njgcqtj',
'njg2i3d',
'njg2gy2',
'njfajh4',
'njf8ssf',
'njexvb7',
'njduq13',
'njdd81k',
'njbixqz',
'njbgvo2',
]

In [9]:
for award_id in missing_awards:
    manually_award_comment(award_id)
    sleep(60)

Manually awarding points based on comment njp9fgo by u/tilltheendof-time.
Manually awarding points based on comment njo5jzo by u/EmployNo1234.
Manually awarding points based on comment njnu3xv by u/EmployNo1234.
Manually awarding points based on comment njlhnl0 by u/There_And_Back1.
Manually awarding points based on comment njlhfux by u/There_And_Back1.
Manually awarding points based on comment njlat3d by u/There_And_Back1.
Manually awarding points based on comment njl2zrj by u/There_And_Back1.
Manually awarding points based on comment njl10dk by u/There_And_Back1.
Manually awarding points based on comment njjffqm by u/tuomas_samuli_photos.
Manually awarding points based on comment nji1nng by u/Present-Vegetable186.
Manually awarding points based on comment njh6wj5 by u/Present-Vegetable186.
Manually awarding points based on comment njgcqtj by u/Present-Vegetable186.
Manually awarding points based on comment njg2i3d by u/tengopelograsoso.
Manually awarding points based on comment njg2g

# Step 5: Manually look for missed awards that need checking

I would do this by starting at /new, scrolling back to post from the below last-checked comment, and then manually opening all the posts after that and searching for `!CritiquePoint`. Do this until you are fairly certain you've gone far enough.

In [15]:
print('This is the last comment we were able to check:')
print(awarding_comment.permalink)

print('\nComments older than this may need checking manually.')

This is the last comment we were able to check:
/r/photocritique/comments/xza4r1/feedback_request_on_portrait/irmugrd/

Comments older than this may need checking manually.


In [42]:
#  manually_award_comment('irah590')

Manually awarding points based on comment irah590 by u/Koosen-.


In [ ]:
# https://old.reddit.com/r/photocritique/comments/xwqlb5/my_thought_was_better_than_the_execution/irah590/